# `titanic.ipynb`
타이타닉 생존자 데이터 셋

In [ ]:
# -q (quiet) 옵션은 설치 로그를 안보여줌(조용히함)
%pip install -q seaborn

In [ ]:
# 타이타닉 데이터셋 불러오기
# (seaborn 기본 내장된 예제 데이터셋)
import seaborn as sns
import pandas as pd

# titanic 데이터 DataFrame
titanic = sns.load_dataset('titanic')

# 처음 5개 행 보기
print("타이타닉 데이터 미리보기:")
print(titanic.head())

# 데이터셋 정보 확인
print("\n데이터셋 정보:")
print(titanic.info())

# 데이터 요약 통계
print("\n데이터 요약 통계:")
print(titanic.describe())

In [ ]:
# Col별 결측치 함계
titanic.isna().sum()
# 승객 등급 등장 빈도 (1, 2, 3 등석)
titanic['pclass'].value_counts()
# 성별 빈도 (남/녀)
titanic['sex'].value_counts()
# 생존 여부 빈도 (생존자/사망자)
titanic['survived'].value_counts()

In [ ]:
# 성별에 따른 생존율
titanic.groupby('sex')['survived'].mean()
# 더 복잡한 통계 가능
titanic.groupby('sex').agg({'survived': ['mean']})

# 승객 등급에 따른 생존율
titanic.groupby('pclass')['survived'].mean()

In [ ]:
# 성별 & 승객 등급에 따른 생존
titanic.groupby(['sex', 'pclass'])['survived'].mean()
# 피벗테이블
titanic.pivot_table(
    values='survived',
    index='sex',
    columns='pclass',
    aggfunc='mean'
)

# 나이 그룹별 생존율
titanic['age_group'] = pd.cut(
    titanic['age'],
    bins=[0, 12, 18, 35, 60, 100, 200],  # 초과 ~ 이하
    labels=['아동', '청소년', '청년', '중장년', '노년', '불사']
)

titanic.head()

#  obeserved 옵션 == 카테고리는 있는데, 해당되는 데이터가 없을 때 표시한다 / 안한다. 
titanic.groupby('age_group', observed=False)['survived'].mean()

# 성별 + 나이그룹으로 생존율 확인
titanic.groupby(['sex', 'age_group'])['survived'].mean()
# 피벗테이블
titanic.pivot_table(
    values='survived',
    index='sex',
    columns='age_group',
    aggfunc='mean'
)

In [ ]:
titanic = sns.load_dataset('titanic')
# 결측치 확인
missing = titanic.isna().sum()
# 결측 있는 항목만 확인
missing[missing > 0]

# 결측 비율
missing_p = titanic.isna().mean() * 100
missing_p[missing_p > 0]

In [ ]:
# 결측치 채우기
# age: 중요한 정보 : 평균/중앙값 대체
# embarked: 가장 많은 사람들이 탄 곳으로 대체
# deck: 추측 불가능(의믹 없음) -> 삭제

# 필요하다면, 카피떠서 진행
titanic_processed = titanic.copy()

# 비어있던 행 마스킹
age_mask = titanic['age'].isna()

# 나이 결측치 채우기 (남녀상관없이 전체평균으로 채움)
mean_age = titanic['age'].mean()
titanic_processed['age'] = titanic['age'].fillna(mean_age)

# 결측치 있는지 재확인
titanic_processed.isna().sum()

# 평균으로 채워진 값들만 확인
titanic_processed.loc[age_mask]


In [ ]:
# 전체평균으로 대체하는게 옳을까..?
# 성별/객실별로 평균을 추정하는건?

# 모든 사람들을 성별/객실 그룹의 평균 나이로 바꾼 Series
print(titanic.groupby(['sex', 'pclass'])['age'].mean())
mean_ages = titanic.groupby(['sex', 'pclass'])['age'].transform('mean')

# titanic 의 age col 중 빈 값만, mean_ages로 채움.
titanic_processed['age'] = titanic['age'].fillna(mean_ages)

# 원래 비어있던 애들만 확인 
titanic_processed.loc[ titanic['age'].isna() ]

In [ ]:
# embarked (탑승 도시) -> 최빈값(가장 많이 탄 곳)

# 비어있는 항구 개수
titanic['embarked'].isna().sum()

# 최빈값 (Series - 벡터)
mode_embarked = titanic['embarked'].mode()[0]  # Series 에서 1개 뽑기
# 빈 값은 최빈값으로 교체
titanic_processed['embarked'] = titanic['embarked'].fillna(mode_embarked)

# 처리 완료 확인 (빈값 0개)
titanic_processed['embarked'].isna().sum()

In [ ]:
# deck 은 77%가 비어있음 -> 삭제
titanic['deck'].isna().mean() * 100  # 빈값 비율

# 'deck' 컬럼은 삭제  - inplace 는 실행하면, 2번째 실행 불가능
titanic_processed.drop('deck', axis=1, inplace=True)

In [ ]:
titanic_processed.info()

## 타이타닉 이상치 처리 (~15:35)

In [ ]:
# 타이타닉 데이터셋 불러오기
# (seaborn 기본 내장된 예제 데이터셋)
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# titanic 데이터 DataFrame
titanic = sns.load_dataset('titanic')

# 수치 데이터에만 집중
numeric_cols = ['age', 'fare']
print("타이타닉 데이터셋 기본 정보:")
print(titanic[numeric_cols].describe())

In [ ]:
# 결측치(Missing Value) 처리 (이전에 배운 내용 활용)
# 모든 사람들을 성별/객실 그룹의 평균 나이로 바꾼 Series
tp = titanic.copy()

print(titanic.groupby(['sex', 'pclass'])['age'].mean())

# 각 승객의 그룹에 맞춰서(성별 + 객실등급) 해당 그룹의 평균으로 일괄 변경
mean_ages = titanic.groupby(['sex', 'pclass'])['age'].transform('mean')
mean_ages
# titanic 의 'age' col 중 빈 값만, mean_ages로 채움.
tp['age'] = titanic['age'].fillna(mean_ages)


In [ ]:
# 결측치 처리 안하면 제대로 시각화 제대로 안나옴

# 1. 이상치 탐지 및 시각화
plt.figure(figsize=(15, 10))

# 나이 분포
plt.subplot(2, 4, 1)
plt.hist(tp['age'], bins=30, alpha=0.7)
plt.title('나이 분포')
plt.xlabel('나이')

plt.subplot(2, 4, 2)
plt.boxplot(tp['age'])
plt.title('나이 박스플롯')

# 요금 분포
plt.subplot(2, 4, 3)
plt.hist(tp['fare'], bins=50, alpha=0.7)
plt.title('요금 분포')
plt.xlabel('요금')

plt.subplot(2, 4, 4)
plt.boxplot(tp['fare'])
plt.title('요금 박스플롯')

# 산점도
plt.subplot(2, 4, 5)
plt.scatter(tp['age'], tp['fare'], alpha=0.6)
plt.xlabel('나이')
plt.ylabel('요금')
plt.title('나이 vs 요금')

# 객실 등급별 요금 분포
plt.subplot(2, 4, 6)
sns.boxplot(x='pclass', y='fare', data=tp)
plt.title('객실 등급별 요금')

# 생존자별 나이 분포
plt.subplot(2, 4, 7)
sns.boxplot(x='survived', y='age', data=tp)
plt.title('생존자별 나이')

# 생존자별 요금 분포
plt.subplot(2, 4, 8)
sns.boxplot(x='survived', y='fare', data=tp)
plt.title('생존자별 요금')

plt.tight_layout()
plt.show()

In [ ]:
tp[numeric_cols].describe()

In [ ]:
# 2. 요금 데이터의 이상치 분석
# IQR 방법으로 이상치 탐지
Q1 = tp['fare'].quantile(0.25)
Q3 = tp['fare'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

fare_outliers = (tp['fare'] < lower) | (tp['fare'] > upper)
# 전체 IQR 기준 이상치 개수 / 전체 개수 (%)
fare_outliers.sum() / fare_outliers.count() * 100

In [ ]:
# 이상치에 해당하는 승객들 특성 분석
outliers_passengers = tp[fare_outliers]
# 객실 등급 분포
print(outliers_passengers['pclass'].value_counts())
# 성별 분포
print(outliers_passengers['sex'].value_counts())
# 생존 여부
print(outliers_passengers['survived'].value_counts())

# 추가로 Grouping 해서 분석도 가능

In [ ]:
# 3. 객실 등급을 고려한 이상치 분석
# 객실 등급별로 요금 이상치 탐지 (전체 기준 이상치가 객실별로는 이상치 아닐 수 있음)

def detect_outliers_by_group(data: pd.DataFrame, group_col: str, value_col: str):
    """그룹별 이상치 탐지"""
    # 우선 전원 모두 이상치 아님(False)
    outliers = pd.Series(False, data.index)
    # 현재 group_col 기준 유니크한 데이터 (pclass -> 1, 2, 3)
    for group in data[group_col].unique():
        # df[df['pclass'] == 3]['fare'] /  df[df['pclass'] == 1]['fare'] / 
        group_data = data[data[group_col] == group][value_col]
        Q1 = group_data.quantile(0.25)        
        Q3 = group_data.quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        group_outliers = (group_data < lower) | (group_data > upper)
        outliers[group_data.index] = group_outliers

    return outliers


In [ ]:
# 객실 등급별 이상한 사람들 (T/F mask series)
fare_outliers_by_pclass = detect_outliers_by_group(tp, 'pclass', 'fare')
# 각 등급별로 따로 선별한 이상치 개수
fare_outliers_by_pclass.sum()

In [ ]:
# 4. 이상치 처리 방법 적용 및 비교
# 방법 1: 제거
titanic_removed = titanic[~fare_outliers_by_pclass].copy()

In [ ]:
# 방법 2: 윈저화 (95% 백분위수로 제한) [전체]
titanic_winsorized = titanic.copy()
upper_limit = titanic['fare'].quantile(0.95)
titanic_winsorized['fare'] = titanic_winsorized['fare'].clip(upper=upper_limit)

In [ ]:
# 방법 3: 객실 등급별 중앙값으로 대체
titanic_replaced = titanic.copy()
for pclass in titanic['pclass'].unique():
    class_data = titanic[titanic['pclass'] == pclass]
    class_median = class_data['fare'].median()
    
    # 해당 등급의 이상치를 중앙값으로 대체
    class_outliers = fare_outliers_by_pclass & (titanic['pclass'] == pclass)
    titanic_replaced.loc[class_outliers, 'fare'] = class_median

In [ ]:
# 5. 처리 결과 비교
print("\n이상치 처리 결과 비교:")
print("=" * 50)

methods = {
    '원본': titanic['fare'],
    '제거': titanic_removed['fare'],
    '윈저화': titanic_winsorized['fare'],
    '대체': titanic_replaced['fare']
}

comparison_stats = pd.DataFrame()
for method, data in methods.items():
    stats_dict = {
        '데이터수': len(data),
        '평균': data.mean(),
        '중앙값': data.median(),
        '표준편차': data.std(),
        '최대값': data.max()
    }
    comparison_stats[method] = stats_dict

print(comparison_stats.round(2))

In [ ]:
# 6. 시각화: 처리 방법별 분포 비교
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.hist(titanic['fare'], bins=50, alpha=0.7, color='blue')
plt.title('원본 요금 분포')
plt.xlabel('요금')

plt.subplot(2, 3, 2)
plt.hist(titanic_removed['fare'], bins=50, alpha=0.7, color='green')
plt.title('이상치 제거 후')
plt.xlabel('요금')

plt.subplot(2, 3, 3)
plt.hist(titanic_winsorized['fare'], bins=50, alpha=0.7, color='orange')
plt.title('윈저화 후')
plt.xlabel('요금')

plt.subplot(2, 3, 5)
plt.hist(titanic_replaced['fare'], bins=50, alpha=0.7, color='purple')
plt.title('이상치 대체 후')
plt.xlabel('요금')

plt.subplot(2, 3, 6)
# 박스플롯으로 비교
box_data = [
    titanic['fare'].values,
    titanic_removed['fare'].values,
    titanic_winsorized['fare'].values,
    titanic_replaced['fare'].values
]
plt.boxplot(box_data, labels=['원본', '제거', '윈저화', '대체'])
plt.title('처리 방법별 박스플롯 비교')
plt.ylabel('요금')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 7. 이상치 처리가 분석에 미치는 영향 확인
correlation_comparison = pd.DataFrame({
    '원본': [titanic['fare'].corr(titanic['survived'])],
    '제거': [titanic_removed['fare'].corr(titanic_removed['survived'])],
    '윈저화': [titanic_winsorized['fare'].corr(titanic_winsorized['survived'])],
    '대체': [titanic_replaced['fare'].corr(titanic_replaced['survived'])]
}, index=['요금-생존율 상관계수'])

print("\n이상치 처리가 상관관계에 미치는 영향:")
print(correlation_comparison.round(4))

## EDA 실습 w/ titanic

In [ ]:
# 타이타닉 데이터셋 불러오기
# (seaborn 기본 내장된 예제 데이터셋)
import seaborn as sns
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (선택사항)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.style.use('default')

# titanic 데이터 DataFrame
titanic = sns.load_dataset('titanic')

## 분석 목표
- 타이타닉 승객의 생존에 영향을 준 주요 요인 파악
- 생존율 개선을 위한 안전 정책 수립 근거 마련
- 위험 그룹 식별 및 우선 구조 대상 결정

### 핵심 질문
1. 어떤 승객 특성이 생존에 가장 큰 영향을 미쳤는가?
2. 사회 경제적 지위가 생존에 영향을 미쳤는가?
3. 나이/성별에 따른 생존 패턴은 어떠한가?

In [ ]:
# 1. 기본 정보 파악(데이터 개요)
# 처음 5개 행 보기
print("타이타닉 데이터 미리보기:")
print(titanic.head())

# 데이터셋 정보 확인
print("\n데이터셋 정보:")
print(titanic.info())

# 데이터 요약 통계
print("\n데이터 요약 통계:")
print(titanic.describe())

# 메모링 사용량
print('\n메모리 사용량:')
print(titanic.memory_usage(deep=True).sum() / 1024 , 'mb')

In [ ]:
# 2. 생존률 기본 분석
print('---생존률 기본 분석---')
survival_rate = titanic['survived'].mean()
print(f'전체 생존률: {survival_rate:.1%}')

# 생존자/사망자 분포
dead, survivor = titanic['survived'].value_counts()
print(f'생존자: {survivor}, 사망자: {dead}')

# 

---생존률 기본 분석---
전체 생존률: 38.4%
생존자: 342, 사망자: 549
